# Intro to Text Analytics - Homework 2

Group 1:
- Boluwade Alabi
- Elizabeth Burke
- Lilah Koudelka
- Michael Mullen
- Cathal Weakliam

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gather-Data-into-Data-Frame" data-toc-modified-id="Gather-Data-into-Data-Frame-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Gather Data into Data Frame</a></span></li><li><span><a href="#Extract-Tokens-From-Raw-Text" data-toc-modified-id="Extract-Tokens-From-Raw-Text-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extract Tokens From Raw Text</a></span><ul class="toc-item"><li><span><a href="#Transformation-Functions" data-toc-modified-id="Transformation-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Transformation Functions</a></span><ul class="toc-item"><li><span><a href="#Tokenization" data-toc-modified-id="Tokenization-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Tokenization</a></span></li><li><span><a href="#Remove-Punctuation-and-Special-Characters" data-toc-modified-id="Remove-Punctuation-and-Special-Characters-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Remove Punctuation and Special Characters</a></span></li><li><span><a href="#Decapitalization" data-toc-modified-id="Decapitalization-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Decapitalization</a></span></li><li><span><a href="#Remove-Numbers" data-toc-modified-id="Remove-Numbers-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Remove Numbers</a></span></li><li><span><a href="#Lemmatization" data-toc-modified-id="Lemmatization-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lemmatization</a></span></li><li><span><a href="#Remove-Stop-Words" data-toc-modified-id="Remove-Stop-Words-2.1.6"><span class="toc-item-num">2.1.6&nbsp;&nbsp;</span>Remove Stop Words</a></span></li></ul></li><li><span><a href="#Tests" data-toc-modified-id="Tests-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Tests</a></span><ul class="toc-item"><li><span><a href="#Test-Cleaned-Data" data-toc-modified-id="Test-Cleaned-Data-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Test Cleaned Data</a></span></li><li><span><a href="#Do-Tokenization-in-One-Go" data-toc-modified-id="Do-Tokenization-in-One-Go-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Do Tokenization in One Go</a></span></li></ul></li><li><span><a href="#Save-Data-Frame-to-CSV-File" data-toc-modified-id="Save-Data-Frame-to-CSV-File-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Save Data Frame to CSV File</a></span></li></ul></li><li><span><a href="#Token-Analysis" data-toc-modified-id="Token-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Token Analysis</a></span><ul class="toc-item"><li><span><a href="#Most-Common-N-Grams" data-toc-modified-id="Most-Common-N-Grams-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Most-Common N-Grams</a></span><ul class="toc-item"><li><span><a href="#Word-Cloud" data-toc-modified-id="Word-Cloud-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Word Cloud</a></span></li><li><span><a href="#Bar-Charts" data-toc-modified-id="Bar-Charts-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Bar Charts</a></span></li></ul></li><li><span><a href="#Category-Frequency-and-Balance" data-toc-modified-id="Category-Frequency-and-Balance-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Category Frequency and Balance</a></span></li><li><span><a href="#Distribution-of-Document-Lengths" data-toc-modified-id="Distribution-of-Document-Lengths-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Distribution of Document Lengths</a></span></li></ul></li><li><span><a href="#TFIDF-Vectorisation" data-toc-modified-id="TFIDF-Vectorisation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TFIDF Vectorisation</a></span><ul class="toc-item"><li><span><a href="#Unit-Testing" data-toc-modified-id="Unit-Testing-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Unit Testing</a></span></li></ul></li><li><span><a href="#TFIDF-Vector-Analysis" data-toc-modified-id="TFIDF-Vector-Analysis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>TFIDF Vector Analysis</a></span><ul class="toc-item"><li><span><a href="#Category-Similarity" data-toc-modified-id="Category-Similarity-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Category Similarity</a></span></li><li><span><a href="#Most-Important-Words" data-toc-modified-id="Most-Important-Words-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Most Important Words</a></span><ul class="toc-item"><li><span><a href="#Across-Entire-Corpus" data-toc-modified-id="Across-Entire-Corpus-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Across Entire Corpus</a></span></li><li><span><a href="#Within-Each-Category" data-toc-modified-id="Within-Each-Category-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Within Each Category</a></span></li></ul></li></ul></li><li><span><a href="#Initial-Model-Building" data-toc-modified-id="Initial-Model-Building-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Initial Model Building</a></span></li><li><span><a href="#Model-#1:-KNN" data-toc-modified-id="Model-#1:-KNN-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Model #1: KNN</a></span><ul class="toc-item"><li><span><a href="#Discussion" data-toc-modified-id="Discussion-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Discussion</a></span></li><li><span><a href="#Hyperparameter-Optimization" data-toc-modified-id="Hyperparameter-Optimization-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Hyperparameter Optimization</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Accuracy-and-Error-Rate" data-toc-modified-id="Accuracy-and-Error-Rate-7.4.1"><span class="toc-item-num">7.4.1&nbsp;&nbsp;</span>Accuracy and Error Rate</a></span></li><li><span><a href="#Confusion-Matrix" data-toc-modified-id="Confusion-Matrix-7.4.2"><span class="toc-item-num">7.4.2&nbsp;&nbsp;</span>Confusion Matrix</a></span></li><li><span><a href="#F1-Score" data-toc-modified-id="F1-Score-7.4.3"><span class="toc-item-num">7.4.3&nbsp;&nbsp;</span>F1 Score</a></span></li></ul></li></ul></li><li><span><a href="#Model-#2:-Naive-Bayes" data-toc-modified-id="Model-#2:-Naive-Bayes-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Model #2: Naive Bayes</a></span><ul class="toc-item"><li><span><a href="#Discussion" data-toc-modified-id="Discussion-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Discussion</a></span></li><li><span><a href="#Hyperparameter-Optimization" data-toc-modified-id="Hyperparameter-Optimization-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Hyperparameter Optimization</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Accuracy-and-Error-Rate" data-toc-modified-id="Accuracy-and-Error-Rate-8.4.1"><span class="toc-item-num">8.4.1&nbsp;&nbsp;</span>Accuracy and Error Rate</a></span></li><li><span><a href="#Confusion-Matrix" data-toc-modified-id="Confusion-Matrix-8.4.2"><span class="toc-item-num">8.4.2&nbsp;&nbsp;</span>Confusion Matrix</a></span></li><li><span><a href="#F1-Score" data-toc-modified-id="F1-Score-8.4.3"><span class="toc-item-num">8.4.3&nbsp;&nbsp;</span>F1 Score</a></span></li></ul></li></ul></li><li><span><a href="#Model-#3:-Logistic-Regression" data-toc-modified-id="Model-#3:-Logistic-Regression-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Model #3: Logistic Regression</a></span><ul class="toc-item"><li><span><a href="#Discussion" data-toc-modified-id="Discussion-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Discussion</a></span></li><li><span><a href="#Hyperparameter-Optimization" data-toc-modified-id="Hyperparameter-Optimization-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Hyperparameter Optimization</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Accuracy-and-Error-Rate" data-toc-modified-id="Accuracy-and-Error-Rate-9.4.1"><span class="toc-item-num">9.4.1&nbsp;&nbsp;</span>Accuracy and Error Rate</a></span></li><li><span><a href="#Confusion-Matrix" data-toc-modified-id="Confusion-Matrix-9.4.2"><span class="toc-item-num">9.4.2&nbsp;&nbsp;</span>Confusion Matrix</a></span></li><li><span><a href="#F1-Score" data-toc-modified-id="F1-Score-9.4.3"><span class="toc-item-num">9.4.3&nbsp;&nbsp;</span>F1 Score</a></span></li></ul></li></ul></li><li><span><a href="#Ensembling" data-toc-modified-id="Ensembling-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Ensembling</a></span><ul class="toc-item"><li><span><a href="#Voting-Ensemble" data-toc-modified-id="Voting-Ensemble-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Voting Ensemble</a></span></li><li><span><a href="#Bagging-Ensemble" data-toc-modified-id="Bagging-Ensemble-10.2"><span class="toc-item-num">10.2&nbsp;&nbsp;</span>Bagging Ensemble</a></span></li><li><span><a href="#Bagging-Ensemble-with-Feature-Subspacing" data-toc-modified-id="Bagging-Ensemble-with-Feature-Subspacing-10.3"><span class="toc-item-num">10.3&nbsp;&nbsp;</span>Bagging Ensemble with Feature Subspacing</a></span></li><li><span><a href="#Boosting-Ensemble" data-toc-modified-id="Boosting-Ensemble-10.4"><span class="toc-item-num">10.4&nbsp;&nbsp;</span>Boosting Ensemble</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-10.5"><span class="toc-item-num">10.5&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li><li><span><a href="#Applying-the-Final-Model-to-the-Test-Set" data-toc-modified-id="Applying-the-Final-Model-to-the-Test-Set-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Applying the Final Model to the Test Set</a></span></li></ul></div>

In [ ]:
import collections
import math
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
import sklearn
import wordcloud as wc
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import RBF
from sklearn.linear_model import LogisticRegression

# Use a random seed to ensure results are consistent between runs
np.random.seed(0)

## Gather Data into Data Frame

We read the CSV files into a Pandas DataFrame object so we can easily analyse the data.

In [ ]:
df_handle = pd.read_csv("trainingset.csv", sep="^", header=0)
df_handle.head()

Now that we have loaded our data into a DataFrame, we will do some cleaning and pre-processing on the data contained within.

## Extract Tokens From Raw Text

Text data is unstructured, and comes in many different forms. Before we can apply models to it we need to preprocess and normalize the data into a fixed-length vector representation.

To do this, we first tokenize the documents - i.e. split them into a list of words, or _tokens_. After tokenizing, we will perform some data cleaning and transformation (like removing special characters and converting every word to lowercase).

We define each of our transformation functions individually so we can discuss and test them separately. We also define a function, `normalize_text()`, that performs all of the transformation functions on a Pandas Series.

### Transformation Functions

Firstly, we define some utility functions that are needed by the transformation functions.

In [ ]:
def get_stop_words():
    """
    Returns a list of stopwords that should be removed when preprocessing text.
    Included are nltk stopwords, salutations and a list of stopwords shown in the labs.
    """
    stop_words = nltk.corpus.stopwords.words('english')

    # Add salutations to the stop words list.
    salutations = ['mr','mrs','mss','dr','phd','prof','rev']
    stop_words.extend(salutations)

    additional_stop_words = ["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]
    stop_words.extend(additional_stop_words)

    return stop_words


def pos_to_wordnet_tag(pos_tag):
    """
    Convert an NLTK.pos_tag to a WordNet tag that the lemmatizer uses.
    """
    if pos_tag.lower().startswith('j'):
        return 'a'
    elif pos_tag.lower().startswith('v'):
        return 'v'
    elif pos_tag.lower().startswith('n'):
        return 'n'
    elif pos_tag.lower().startswith('r'):
        return 'r'
    else:
        # Default POS for lemmatization is noun.
        return 'n'

Now we define the transformation functions themselves.

In [ ]:
def remove_hyphens(text):
    """
    Takes a string of text and returns a string of text with hyphens deleted.
    We do this so words like "e-mail", "wi-fi", etc. are kept together.
    """
    return text.replace("-", "")


def tokenize(text):
    """
    Takes a string of text and returns a list of string tokens.
    """
    # Pattern matches one or more alphanumeric characters (or underscores).
    TOKENIZER_REGEX = r'\w+'
    tokenizer = nltk.tokenize.RegexpTokenizer(TOKENIZER_REGEX)  
    return tokenizer.tokenize(text)


def decapitalize(tokens):
    """
    Takes a list of tokens and returns a list of tokens, with every token in lowercase form.
    """
    # Map the string to-lowercase method to every value in the series.
    return [token.lower() for token in tokens]


def remove_numbers(tokens):
    """
    Takes a list of tokens and returns a list of tokens that do not consist solely of numeric characters.
    For example, "3" is removed, but not "3G" or "Three".
    """
    return [token for token in tokens if not token.isnumeric()]


def remove_special_chars(tokens):
    """
    Takes a list of tokens and returns that list, without special characters.
    
    The only special characters that are left in the text are underscores since we used
    the regex '\w+'.
    """    
    return [word.replace("_", "") for word in tokens]


STOP_WORDS = get_stop_words()

def remove_stop_words(tokens):
    """
    Takes a list of tokens and returns a list of those tokens that are not contained 
    in the stop words list.
    """
    return [token for token in tokens if token not in STOP_WORDS]
    
    
def lemmatize(tokens):   
    """
    Takes a list of tokens and returns a list of the same tokens, transformed using the 
    WordNet Lemmatizer. This lemmatization process converts words into a common root.
    For example, it converts plural words into singular form, or past-tense verbs into their root.
    """
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    lemmatized_words = []
    for word, tag in nltk.pos_tag(tokens):
        wordnet_tag = pos_to_wordnet_tag(tag)
        lemmatized_words.extend([lemmatizer.lemmatize(word, wordnet_tag)])
        
    return lemmatized_words


def normalize_text(series, keep_stop_words=False, lemmatization=True):
    """ 
    Takes a pandas Series object and returns a list of tokens for that series.
    Gives an option (default=True) on whether to do lemmatization or not.
    """
    newseries = (series.apply(remove_hyphens)
                  .apply(tokenize)
                  .apply(remove_special_chars)
                  .apply(decapitalize)
                  .apply(remove_numbers))
    
    # Repeat removal of numbers after lemmatization. This is discussed below.
    if lemmatization:
        newseries = newseries.apply(lemmatize)
        newseries = newseries.apply(remove_numbers)
    
    if not keep_stop_words:
        newseries = newseries.apply(remove_stop_words)

    return newseries

We now go through applying each transformation function step-by-step.

#### Tokenization

Our first step is to tokenize the words in the text of each article. We will create a new column in our dataframe and call it `tokens`, filling it with the result of applying our `tokenize()` function on the `contents` column.

We define a token using the regular expression `\w+`, which matches any alphanumeric character or an underscore, "_".

In [ ]:
df_handle['tokens'] = df_handle['content'].apply(remove_hyphens).apply(tokenize)
df_handle.head()

We see that we have successsfully split the `content` column into its component words.

#### Remove Punctuation and Special Characters

Because of the regex pattern that we used, `\w+`, we have already separated tokens into only alphanumeric words. This means that we don't have to do any further filtering to remove punctuation and special characters. We also remove any tokens that are just empty strings.

In [ ]:
df_handle['tokens'] = df_handle['tokens'].apply(remove_special_chars)
df_handle['tokens'].head()

#### Decapitalization

Our data is still not fully normalized though. We want all tokens to have the same form, but some of the tokens are uppercase, some lowercase, and some a mixture of both. To ensure consistency in our data, we are going to convert every token into its lowercase form.

In [ ]:
df_handle['tokens'] = df_handle['tokens'].apply(decapitalize)
df_handle['tokens'].head()

#### Remove Numbers

We can see that some of our tokens are just numeric strings, which we want to remove.

Note that we will not remove tokens that contain both letters and numbers. For example, "3g" considered a word and it would lose its meaning if it were changed to just "g".

In [ ]:
df_handle['tokens'] = df_handle['tokens'].apply(remove_numbers)
df_handle['tokens'].head()

#### Lemmatization

Consider the second document. We have tokens like "games" and "pixels" - they are plural nouns. However, if we want to count the occurrences of these tokens, they should not be considered as different words to their singular counterparts. 

In [ ]:
first_article = df_handle.tokens.iloc[1].copy()

In [ ]:
count_games = first_article.count("games")
count_game = first_article.count("game")

print("Count of 'games' = {}".format(count_games))
print("Count of 'game'  = {}".format(count_game))

We perform an operation known as *lemmatization*, which takes a word and alters it to its root form. For example, it will convert plural nouns to singular nouns, or convert past-tense verbs to infinitive verbs.

Note: This cell takes ~30 seconds to run.

In [ ]:
df_handle['tokens'] = df_handle['tokens'].apply(lemmatize)
df_handle['tokens'].head()

Let's look at some of the lemmatizer results.

In [ ]:
" ".join(df_handle.tokens.iloc[1])

We look to see if this has grouped the words 'game' and 'games':

In [ ]:
count_game = df_handle['tokens'].iloc[1].count("game")
print("Count of 'game'  = {}".format(count_game))

It has, so our lemmatization has been successful.

However, we notice that numbers have reappeared in our token set. This is because our `remove_numbers()` function didn't remove a word like "20s" because it wasn't all numeric, however, it was lemmatized back to "20".

In [ ]:
df_handle['tokens'].iloc[1].count("20")

In [ ]:
lemmatize(['20s'])

To combat this, we are going to rerun our `remove_numbers()` function after having applied lemmatization.

In [ ]:
df_handle['tokens'] = df_handle['tokens'].apply(remove_numbers)
df_handle['tokens'].iloc[1].count("20")

#### Remove Stop Words

We have now cleaned up our tokens so that they only contain real words and in lemmatized form. However, some words are not as valuable as others. For example, if we want to calculate the most-common word in an article, it is likely that it will be a word like 'a', 'the', etc. These words are called *stop words*. They do not gives us valuable information since they are so prevalent in the English language. So we decided that we are going to remove them from our list of tokens.

In [ ]:
df_handle['tokens'] = df_handle['tokens'].apply(remove_stop_words)
df_handle['tokens'].head()

### Tests
#### Test Cleaned Data

It is very important that we run tests to ensure our data is as we expect it to be. Failing to carry out tests can lead to problems in analysis further on.

We define and run a test now to ensure that our data has been tokenized successfully and those tokens are cleaned and normalized as we expect.

In [ ]:
def column_is_clean(series):
    return all([tokens_are_clean(tokens) for tokens in series])
   

def tokens_are_clean(tokens):    
    if any([token.isupper() for token in tokens]):
        print("Some tokens are not uppercase")
        return False
    
    elif any([not token.isalnum() for token in tokens]):
        print("Some tokens contain special (non-alphanumeric) characters")
        return False
    
    elif any([token.isnumeric() for token in tokens]):
        print("Some tokens are numeric")
        return False
    
    elif any([True for token in tokens if token in get_stop_words()]):
        print("Some stop words were not removed")
        return False
    
    else:
        # Data has been cleaned successfully.
        return True

In [ ]:
# Throw an error if test fails.
assert column_is_clean(df_handle['tokens']), "The data is not clean!"
print("Data Cleaned and Tokenized Successfully!")

#### Do Tokenization in One Go

Now that we have shown that our data has been successfully cleaned, we also need to test that our function to do the normalization in one go (which we will use later) is equivalent to what we have done step by step. 

In [ ]:
oldtokens = df_handle['tokens'].copy()
df_handle['tokens'] = normalize_text(df_handle['content'])

# Throw an error if not equal.
assert oldtokens.equals(df_handle['tokens']), "The tokens are not equal!"

print("The one-by-one and all-at-once methods are equal!")

### Save Data Frame to CSV File

Now that we have read and processed our data, we export it to a CSV file. 

In [ ]:
df_handle.to_csv("h2_tokens.csv", sep='^', index=None)

To show that our dataframe was written to a CSV file correctly, we will read it back and compare.

In [ ]:
reread_data = pd.read_csv("h2_tokens.csv", sep="^")
# After reading in the DataFrame from csv, each list of tokens is represented as a string.
# We use `apply` to convert these to Python lists
reread_data['tokens'] = reread_data['tokens'].apply(eval)

# Throw an error if not equal.
assert reread_data.equals(df_handle), "The two data frames are not the same!"
print("The two data frames are equal!")

<a id='token_analy'></a>
## Token Analysis
Now we turn our attention to exploring and analysing the data that we have cleaned, transformed, tokenized and saved into our DataFrame. In particular, we will investigate the tokens columns that we have constructed. We will look at the following features:
- Most-Common N-Grams
- Category Frequency and Imbalance
- Distribution of Document Lengths

### Most-Common N-Grams
We start our analysis by look at the most common words and phrases in the corpus. This will give us a better feel for the data that we are dealing with and may give us some useful insights into  particular words or phrases our classifier should pay most attention to.

We will also analyse the best ways to visualise our results, comparing word clouds with simple bar charts.

#### Word Cloud
We explore our corpus by looking at which words and phrases are most common. One popular way of doing this is by creating a word cloud from the corpus data.

We create the wordcloud image using the `wordcloud` library and plot it. 
<br />
We are also saving it to a file called *corpus_wordcloud.png*.

In [ ]:
# Get all the tokens from our data.
all_tokens_list = [token for tokens in df_handle['tokens'] for token in tokens]

# The word cloud uses a single string not a list, so we convert our 
# list of tokens to a single string.
all_tokens_string = " ".join(all_tokens_list)

# Create the WordCloud object from the string of tokens.
wordcloud = wc.WordCloud(width=1600, height=800).generate(all_tokens_string)

plt.figure(figsize=(20,10), facecolor='k')
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

# Save the word cloud image to a file.
wordcloud.to_file("images/corpus_wordcloud.png")

# Delete this to clean up our notebook.
del all_tokens_list

We can see from this image that "year", "make", and "people" are some of the most common words in the corpus. However it's hard to draw many more conclusions than that from the word cloud. There are a number of problems associated with it:
- Some words appear larger just because they are longer, not because they are more frequent. For example, the word "government" stands out more than other shorter words that have roughly the same frequency, e.g. "uk".

- It is quite hard to judge the font size of different words, especially if the difference between them is small or they are far apart in the word cloud. For instance, which word is more common, "market" or "world"?
- All of the words that are plotted are unigrams. Word clouds don't take bigrams, trigrams, etc. into account, so we lose information we could potentially gain from exploring them. For example, "labour" and "party" appear to have similar frequencies, but perhaps they are part of the bigram referring to the Labour Party, a political organisation? We have no way of telling from this plot.

For these reasons, we don't believe word clouds are an effective way of analysing our dataset. Instead, we will use  simple bar charts, that make it easier to tell the frequency of a particular term and to compare between different terms.

#### Bar Charts
Having shown that word clouds are not the best method for visualising our data, we move on to creating bar charts. To create a bar chart, we need to get the frequency data for unigrams, bigrams, trigrams, etc. We do this by using the `FreqDist` object from the `nltk` library.

The steps we follow are:
- Extract n-grams (for a given n) from our list of tokens
- Create a frequency distribution from the dataset for the n-grams
- Convert this frequency distribution to a DataFrame which we can plot
- Plot the data in the data frame

In [ ]:
def freq_dist_to_df(freq_dist, top_n):
    """
    Convert a frequency distribution to a Data Frame, with columns 'term' and 'count'.
    """
    most_common_terms = []
    
    for term, count in freq_dist.most_common(top_n):
        most_common_terms.append({'term': term, 'count': count})
    
    most_common_df = pd.DataFrame(most_common_terms)
    most_common_df.sort_values(by='count', ascending=True, inplace=True)
    
    # Change term labels from being "(x, y, z)" to "x y z".
    join_tuple = lambda terms: " ".join(terms)
    most_common_df.term = most_common_df.term.apply(join_tuple)

    return most_common_df


def plot_frequency_df(df, title, xlabel="Frequency", ylabel="", fileName=None):
    """
    Plot a horizontal bar chart of the most common words in a given 
    dataframe, with columns 'term' and 'count'. The bars are sorted 
    so that the most-common word appears at the top.
    """
    fig, axis = plt.subplots()

    df.plot.barh(x='term', y='count', ax=axis, legend=None)
    
    axis.set_title(title)
    axis.set_xlabel(xlabel)
    axis.set_ylabel(ylabel)

    plt.tight_layout()

    # Save figure to an image file.
    if fileName is not None:
        plt.savefig("images/" + fileName)

    # This should only be executed after plt.savefig(),
    # otherwise the image saved will be blank.
    plt.show()


def plot_ngrams(ngrams_data, n=1, file_name=None):
    """
    Plot the frequency of n-grams in a Series, for a given n.
    """
    # Get all the tokens for the corpus in a list.
    all_tokens_list = [token for tokens in ngrams_data for token in tokens]

    ngrams = nltk.ngrams(all_tokens_list, n)

    # Get the frequency of each term in the corpus and create a DataFrame from it.
    ngram_frequency = nltk.FreqDist(ngrams)
    ngrams_df = freq_dist_to_df(ngram_frequency, 5)
    
    ngram_name = "Word"  # default
    if n == 2:
        ngram_name = "Bigram"
    elif n == 3:
        ngram_name = "Trigram"

    plot_frequency_df(ngrams_df, title="Top-10 Most-Common %ss for Entire Corpus" % ngram_name,
                      xlabel="Frequency", ylabel=ngram_name, fileName=file_name)

In [ ]:
plot_ngrams(df_handle.tokens, n=1, file_name="most-common-unigram.png")

The most common words for the entire corpus don't give us a huge amount of insight about the data. The most common word is "year" but this could be used in any of the contexts of the document categories. Similarly for the words "make", "people" and "time". However, one word that does stand out is "game", which we would primarily associate with sport. However, it is possible that this word is applicable to other categories too, for example video games fall under the topic of technology.

In [ ]:
plot_ngrams(df_handle.tokens, n=2, file_name="most-common-bigram.png")
plot_ngrams(df_handle.tokens, n=3, file_name="most-common-trigram.png")

We get more information from analysing the bigrams and trigrams than the single words. For example, we see that the some of the top bigrams and trigrams are "bbc news", "bbc news website" and "bbc radio today". This would seem to suggest that the source of our news articles is BBC News. This is also supported by the references to former British Prime Minister Tony Blair, and former British Conservative Party Leader Michael Howard, who are referenced by top bigrams and trigrams too. 

Furthermore, we see that there are two bigrams that stand out for their association with particular document categories. "Prime Minister" is definitely associated with politics and "Chief Executive" is similarly associated with the business topic.

The fifth most common trigram is "million dollar baby". This could be a reference to the boxing film of the same name that won multiple Oscars in 2005. This trigram is probably associated with the entertainment category. However, seeing as it's a film that is related to sports, it could also be associated in some way to the sports category. We will investigate the [overlap between categories and their similarities](#category_similarity) later on in our data analysis.

### Category Frequency and Balance

Now, let's look at the labels that we have been provided with each of the documents in our corpus. There are five possible categories (or *classes*) that the label can represent: sport, entertainment, politics, technology and business. We also refer to our class labels as the *target label* of each document.

We look at how many documents of each category appear in our dataset.

In [ ]:
class_distribution = df_handle.groupby(df_handle.category).category.count()
class_distribution

We normalise these absolute numbers to get a breakdown of the percentage frequency of each category. This makes it easy to compare the figures. We also plot a line, in orange, for the mean frequency, this is the frequency that we expect all categories to have if the data were uniformly distibuted.

In [ ]:
# Convert category frequency to a percentage of all categories.
class_distribution_normalised = (class_distribution / sum(class_distribution)) * 100

# Plot a line representing the mean frequency of all classes.
# This is the frequency each class would have if uniformly distributed.
mean_freq = 100 / len(class_distribution)

fig, axis = plt.subplots()
axis.axhline(y=mean_freq, color='orange')
class_distribution_normalised.plot.bar(ax=axis, rot=0)
axis.set_xlabel("Target Class (Document Category)")
axis.set_ylabel("Frequency (% of Corpus)")
axis.set_title("Target Class Frequencies Across Corpus")

plt.savefig('images/target-class-frequencies.png')
plt.show()

We see that we have a slighly unbalanced target class distribution. This means that not all target labels appear as often as each other. For example, the sport category is most common, whereas the entertainment category is the least common. When building our model, we need to take this into account, to make sure that we are not overly biasing our model based on the distribution of target classes in the training data. In extreme cases, our model could underfit the data and end up predicting the most common label (sport) each time. 

While this would be easy to spot and fix, it is possible that the class distribtion will affect the model in more subtle ways. One method we could use to avoid this would be to use the F-Measure as an evaluation metric for our models instead of just using accuracy. We will investigate this later, in our section on model evaluation.

### Distribution of Document Lengths

Now, we look at some meta-statistics about each document. We compare the length of each document as well as the number of unique words in each document. We breakdown this analysis per target class.

In [ ]:
df_handle['document_length'] = df_handle.tokens.apply(len)

fig, axis = plt.subplots(1, 2, sharey=True)

df_handle.plot.hist(by='document_length', bins=5, ax=axis[0], figsize=(10,4), legend=None)
axis[0].set_xlabel("Document Length (Number of Tokens)")
axis[0].set_title("Document Length Distribution")

clipped_content_length_data = df_handle[(df_handle.document_length > 100) & (df_handle.document_length < 500)]
clipped_content_length_data.plot.hist(by='document_length', bins=5, ax=axis[1], figsize=(10, 4), legend=None)
axis[1].set_xlabel("Document Length (Number of Tokens)")
axis[1].set_title("Document Length Distribution (clipped)")

plt.tight_layout()
plt.savefig("images/document-length-distribution.png")
plt.show()

From the first histogram on the left, we see that the overwhelming majority of documents have between 100 and 500 words. We don't get a huge amount of information from this plot so we replot the data, except that we filter the dataset so that we only include documents whose length is within this interval. This process is known as *clipping*. The clipped data is plotted on the right histogram.

From the second histogram, we get more useful data. We see most of the documents have a length between 100 and 200 characters. It is important that we are aware that the distribution of the lengths of each document is not uniform. 

We also break this down for each document category.

In [ ]:
doc_length_per_class = df_handle.groupby(df_handle.category).document_length.mean()

fig, axis = plt.subplots()

doc_length_per_class.plot.bar(ax=axis, rot=0)

# We plot the average document length for reference.
mean_doc_length = df_handle['document_length'].mean()
axis.axhline(y=mean_doc_length, color='orange')

axis.set_xlabel("Target Class (Document Category)")
axis.set_ylabel("Mean Document Length (num. tokens)")
axis.set_title("Mean Document Length per Target Class")

plt.savefig("images/document-length-per-class.png")
plt.show()

Again, we see that we don't have uniform distribution of document length. This is important to recall, since, when we are using methods to vectorise the documents, if we don't take into account the documents' length, then we will be unfairly biasing longer or shorter documents (depending on the vectorising method). 

For example, if we are calculating the frequency of a term, comparing this across documents that have different lengths is unfair. A word that appears 10 times in a 50 word document is far more prominent that a word that appears 20 times in a 500 word document. So we need to make sure that we normalise our frequency calculation (i.e during TFIDF vectorisation) by the length of the doucment the term appears in. We will return to this point later when we are doing the document vectorisation.

## TFIDF Vectorisation

Our documents are now represented using a list of tokens. However, in order to continue our analysis we will need to transform these documents into numerical vectors. To do this, we will use a TF-IDF Vectorizer.

TF-IDF stands for Term Frequency-Inverse Document Frequency. It is defined as follows:

$$ \text{TF-IDF}(term, document) = \text{TF} \times \text{IDF} $$

$$ \text{IDF}(term) = \log(\frac{n_d + 1}{df(term) + 1}) + 1$$

where:
- $n_d$ is the number of documents in the corpus
- $df(term)$ is the number of documents in which term appears at least once
- $tf(term, document) $ is the number of times the given term appears in the given document

Words that are more common will have a higher TF value. Words that are more significant (that is, they are characteristic to only a small subset of documents), will have a higher IDF value. TF-IDF balances the two in order to find the most-important words in the corpus.

We normalise these vectors by their length that we don't unfairly rank words appearing in shorter documents higher. This is known as the *L2 Norm*:

$$ v_{norm} = \frac{v}{||v||_2} $$

In [ ]:
tfidf_vectorizer = TfidfVectorizer(norm='l2')
document_token_strings = [' '.join(tokens) for tokens in df_handle.tokens]
tfidf_vectors = tfidf_vectorizer.fit_transform(document_token_strings).toarray()

Having created vectors for each of our documents, we now place those vectors into their own DataFrame.

In [ ]:
tfidf_data = pd.DataFrame(
    tfidf_vectors, columns=[tfidf_vectorizer.get_feature_names()], index=df_handle.index)

tfidf_data.head()

### Unit Testing

We do some testing to ensure that our TF-IDF vectorisation was successful. 

First, we define all of the necessary functions for the calculations:

In [ ]:
def term_frequency(term, document, dataframe):
    document_tokens = dataframe.iloc[document].tokens
    frequency = document_tokens.count(term)
    
    return frequency


def inverse_document_frequency(term, dataframe):
    num_documents = len(dataframe)
    
    documents_containing_term = 0
    for document in dataframe.tokens:
        if document.count(term) > 0:
            documents_containing_term += 1
            
    return 1 + math.log((num_documents + 1) / (documents_containing_term + 1))


def raw_tfidf(term, document, dataframe):
    return term_frequency(term, document, dataframe) * inverse_document_frequency(term, dataframe)


def normalised_tfidf(term_tfidf, document_vector):
    return term_tfidf / np.linalg.norm(document_vector)


def tfidf(term, document, dataframe):
    document_tokens = set(dataframe.iloc[document].tokens)

    term_tfidf = raw_tfidf(term, document, dataframe)

    # Use a NumPy array for efficient calculation of the norm.
    document_vector = np.fromiter((raw_tfidf(term, document, dataframe) 
               for term in set(df_handle.iloc[document].tokens)), dtype=float)

    return normalised_tfidf(term_tfidf, document_vector)

We calculate our manual TFIDF value for a pair of (document, term), comparing it with the value from SkLearn's TfidfTokenizer:

In [ ]:
term = "game"
document = 1

# Round TFIDF values to 4 decimal places to account for floating-point imprecision.
manual_tfidf = tfidf(term, document, df_handle)
manual_tfidf = round(manual_tfidf, 4)

sklearn_tfidf = tfidf_data.iloc[document][term]
sklearn_tfidf = round(sklearn_tfidf, 4)

In [ ]:
print("Custom TFIDF: %.4f" % manual_tfidf)
print("SkLearn TFIDF: %.4f" % sklearn_tfidf)

assert manual_tfidf == sklearn_tfidf, "Manual and SkLearn TFIDF values are not equal"

print("Manual and SkLearn TFIDF values are equal!")

We can see that our manual TFIDF value aligns with that from the TfidfVectorizer, so our unit test has been successful.

## TFIDF Vector Analysis

We now perform data analysis on the vectors that we have created for each of the documents. We want to explore trends or features of the documents and their categories to help improve our understanding of them. This will aid us in choosing and evaluating models in later sections.

We are going to analyse the following:
- Category similarity
- Most important words

<!-- Anchor for internal links -->
<a id='category_similarity'></a>

### Category Similarity 

Now we turn our attention to computing which categories are most and least similar to each other. 
We do this with the following steps:
- Take each of our TFIDF document vectors
- Group the documents by their category
- Compute the mean TFIDF vector for each category
- Compute the cosine distance between each mean vector
- Calculate the closest vectors and the vectors furthest apart

We use cosine distance as a measure of how similar each category is. Although distance measures how different each category is from each other, if we invert it then we get a similarity metric. This means that smaller distances imply greater similarity and larger distances mean less similarity. Cosine distance is a useful metric since it doesn't depend on the distance between the vectors in n-dimensional space, which could be unfairly biased by abnormally large vector-components. Instead, it depends on the angle between the vectors.

We start by calculating the mean TFIDF document vectors from the TFIDF matrix and put them into their own dataframe so that we can analyse them further.

In [ ]:
categories = df_handle.category.unique()
mean_vector_df = pd.DataFrame(index=tfidf_data.columns)

for category in categories:
    mean_vector = tfidf_data[df_handle.category == category].mean(axis=0)
    mean_vector_df[category] = mean_vector

In [ ]:
mean_vector_df.tail()

To calculate the distance between the categories, and not between the n-grams in the corpus, we need to transpose our data frame. This swaps the rows and the columns in the data frame. This means that the rows will now correspond to the categories and the columns to the n-grams.

We have done this because it is more efficient to add to a dataframe column-wise. Adding categories' data column-by-column and then transposing is more efficient than if we did the transposition first.

In [ ]:
# Transpose our data frame.
mean_vector_transpose = mean_vector_df.T

# Calculate cosine distance based on the tranposed data.
pairwise_cosine_distance = sklearn.metrics.pairwise_distances(mean_vector_transpose, metric='cosine')

pairwise_cosine_distance

We could read this matrix by eye, but this could get complicated. Instead, we define some helper functions to find the most-similar categories and the least-similar categories.

In [ ]:
def n_closest_categories(array, n):
    """
    Returns a list of tuples of the category-pairs that are most similar to each other.
    The number of tuples returned is decided by the size parameter, n.
    """
    return n_extreme_categories(array, n, multiplier=1)


def n_furthest_categories(array, n):
    """
    Returns a list of tuples of the category-pairs that are furthest away from each other.
    The number of tuples returned is decided by the size parameter, n.
    """
    return n_extreme_categories(array, n, multiplier=-1)


def n_extreme_categories(array, n, multiplier):
    """
    Return a list of tuples of the category pairs that have the most-extreme distances from 
    each other. Extreme can mean either smallest or largest distance from each other. 
    
    In the case of the calculating the largest, we sort our array in reverse. This means that
    we first have to multiply the array by -1, since numpy only provide an ascending-order 
    sorter.
   
    Note that we want to use built-in numpy library functions since they are most-efficient in 
    dealing with numpy arrays.
    """
    
    # We take the indices from 0 to 2n, with a step size of 2. This means that
    # we skip every second index, which will be the same value as the index before it,
    # given that the distance matrix is symmetric.
    # For example, for the largest n=2, we take the first and third maximum elements, 
    # skipping the second since it will be the same as the first.
    extreme_indices = np.argsort(multiplier * array.flatten())[0:2*n:2]

    extreme_categories = []
    
    for ind in extreme_indices:
        two_d_index = np.unravel_index(ind, array.shape)
        
        # Convert the tuple of indices to a tuple of the corresponding categories.
        converted_tuple = tuple(map(lambda c: categories[c], two_d_index))
        extreme_categories.append(converted_tuple)
        
    return extreme_categories

In [ ]:
n_furthest_categories(pairwise_cosine_distance, n=2)

To calculate the most-similar category-pairs we need to remove the zero values from our pairwise-distance array. The values are 0 when comparing a category to itself, but this doens't give us any valuable information. 

Instead, we will copy the pairwise-distance array and transform the zero-values to infinity.

In [ ]:
nonzero_distances = pairwise_cosine_distance.copy()
nonzero_distances[nonzero_distances == 0] = float("inf")

n_closest_categories(nonzero_distances, n=2)

So the two most similar category-pairs are:
- politics and buiness 
- business and tech

And the two least similar category-pairs are:
- sport and business
- entertainment and business

It is important the we recognise and compute these similarities when evaluating our model. At the very least our model should be able to distinguish between the least similar categories of documents. Ideally it will also be able to tell apart the most similar categories. 

However, when evaluating models, it is important to recognise that the category-pairs listed above are the most similar and, therefore, the hardest to tell apart. This means that we can still have a very good, high performance model, even if it is slightly confused between the most similar document categories.

It is also interesting to note that the category of business is included in all of the most and least similar category pairs. This indicates how complex and varied the defining features of particular categories are.

<a id="most-important-words"></a>
### Most Important Words
Now we look at which words are the most important in our dataset. We define importance by the TFIDF metric. These will be the words that are most indicative of a document's category.

#### Across Entire Corpus

First of all, though, let's look at the most important words throughout the entire corpus.

In [ ]:
# Calculate the mean TFIDF vector for the entire corpus.
tfidf_mean = tfidf_data.mean(axis=0)

In [ ]:
# Find the 10 largest TFIDF-valued terms
most_important_words = tfidf_mean.nlargest(10)

most_important_words

#### Within Each Category

Now, we look at the most important words within each category.

Note that we have already calculated the mean TFIDF document vectors for each category in the [Category Similarity](#category_similarity) section above. We will reuse them here.

In [ ]:
fig, axis = plt.subplots(len(categories), sharex=True)

# Keep a list of all terms that appear in the top ten for each category.
most_important_terms = []

for i in range(len(categories)):
    category_vector = mean_vector_transpose.iloc[i]
    
    category_vector.nlargest(10).plot.barh(ax=axis[i], figsize=(10, 15))
    
    most_important_terms.extend(category_vector.nlargest(10).index)
    
    axis_title = "Most Important N-Grams for Category %s"
    axis[i].set_title(axis_title % category_vector.name.title())
    
axis[len(categories) - 1].set_xlabel("Importance (TFIDF Value)")

plt.tight_layout()
plt.savefig("images/most-important-terms-per-category.png")
plt.show()

These bar charts give us a wealth of insight into the defining words for each category. For example, by far the most important term is "film", which is indicative of the entertainment category. This means that a good model will identify the occurences of the word "film" in a document and make it more likely to classify that document as being about entertainment. 

It is also interesting to note that the categories appear to be fairly disjoint from each other, based on their top ten most important words. We can see this by the fact that the only terms that are in the top ten in more than one category are:
- "game", appearing for both the sport and tech categories
- "year", appearing for both the business and entertainment categories

We show this, computationally, below:

In [ ]:
counter = collections.Counter(most_important_terms)

for word in counter:
    if counter[word] > 1:
        print("%s: %s" % (word, counter[word]))

However, when considering all terms, not just the top ten, we have seen that the categories are more aligned. This is discussed in the [Category Similarity section](#category_similarity)  above.

<a id='initial_models'></a>

## Initial Model Building

Sklearn provides many different models for classification. Rather than try to guess which ones would be best for our problem, we will test and analyse a large selection of them. We will then pick the three best models to do further analysis on. 

In order to thoroughly test our models, we will split our data into two sets:
- a *training set* that the models are built using; and
- a *test set* that allows us to assesss the performance of a model on unseen data

We will test each of the models by using cross-validation and using accuracy as a scoring metric. We don't use the F-measure to assess our models yet, since there is a quirk in Sklearn. The F-measure is set to 0 when a model is making predictions on a test set that doesn't contain all of the categories at least once.

However, we do want to use the F-measure later on in our evaluations, so we ensure that the test split of our data contains at least every category at least once.

We perform this train-test split now:

In [ ]:
num_categories = len(df_handle.category.unique())

keep_going = True
while keep_going:
    # Put \ at the end of the line so the statement is continued over the next line.
    train_input_vectors, test_input_vectors, train_labels, test_labels = \
        train_test_split(tfidf_vectors, df_handle['category'])
        
    # If we've found a test split that contains all of the labels 
    # (thus allowing us to do F-measure evaluations), then we stop.
    if len(test_labels.unique()) == num_categories:
        keep_going = False

The training and cross-validation process below takes a long time. Results from a previous execution are included in a markdown cell below. By default, the process is set not to run again. If you would like to run the process, set the constant below to true. 

In [ ]:
RUN_ALL_MODELS_EVAL = False

Note that if you do re-run this process, your result may differ slightly from those in the markdown cell due to the nondeterministic splitting of the dataset during the cross validation step.

In [ ]:
names = [
    "Nearest Neighbors",
    "Naive Bayes",
    "Logistic Regression",
    "AdaBoost",
    "Random Forest",  
    "Linear SVM", 
    "RBF SVM",
    "Decision Tree",
]

classifiers = [
    KNeighborsClassifier(6),
    MultinomialNB(alpha=1.0),
    LogisticRegression(multi_class='multinomial', solver='newton-cg'),
    AdaBoostClassifier(),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1), 
    DecisionTreeClassifier(max_depth=5),
]

if RUN_ALL_MODELS_EVAL:
    for name, classifier in zip(names, classifiers):    
        k_folds = 10
        scoring = 'accuracy'
        scores = cross_val_score(classifier, train_input_vectors, train_labels, 
                                 cv=k_folds, scoring=scoring)

        average_accuracy = np.mean(scores)

        print("%s ave. %s = %.4f" % (name, scoring, average_accuracy))

The results of a previous execution of this process were:

| Model | Accuracy |
| --- | --- |
| Logistic Regression | 0.9735 |
| Naive Bayes | 0.9605 |
| Nearest Neighbors | 0.9347 |
| RBF SVM | 0.9149 |
| AdaBoost ave. |0.6907 |
| Decision Tree | 0.6236 |
| Random Forest | 0.3128 |
| Linear SVM | 0.2374 |

We see that the highest-performing models are the Logistic Regression, Naive Bayes and Nearest Neighbour classifiers. We will now do further investigation and analysis on these models in order to find the optimal model for our dataset.

<a id='m1'></a>

## Model #1: KNN

### Discussion

K-Nearest Neighbours (KNN) is a simple, lazy learning algorithm. It classifies new cases based on their distance to existing training cases. The most-popular choice for the distance function is Euclidean Distance, $d$, defined by:

$$ d(x, y) = \sqrt{\left(x_1 - y_1 \right)^2 + \left(x_2 - y_2 \right)^2 + \dotsc + \left(x_n - y_n \right)^2} $$

where:
- $x$, $y$ are document vectors
- each $x_i$ or $y_i$ is the i<sup>th</sup> feature of document $x$ or $y$.

When classifying an unseen example, KNN:
- Finds the neighbourhood of training examples closest to the unseen example, based on the distance function. The size of the neighbourhood, K, and the distance function are both hyperparameters for the algorithm.
- A prediction is generated based on the categories of all the examples in the neighbourhood. This can be done in two ways:
    1. simple majority voting - the most frequent label is chosen as the prediction (every neighbour has equal weight)
    2. weighted voting - the label from each neighbour is multiplied by the inverse of its distance from the unseen example. This means that the closest neighbours have more of an impact. The label with the highest total vote is chosen as the prediction.
    
The main advantage of KNN is that it is very easy to implement and quick to run. Despite its simplicity, it has been shown to perform well in a wide variety of areas. 

However, when the neighbourhood size, K, is too small, KNN is quite sensitive to noise and outliers. When K is too big, KNN can end up always predicting the majority class. This can be a problem when you have a dataset with unbalanced classes.

<a id='m1_optimize'></a>

### Hyperparameter Optimization

Here, we begin the hyperparameter optimization process in order to choose the best value for K. After choosing the top 3 models, we want to choose the best parameters for each of those models to get the best results. By tuning the parameters, we will find the best value of K that will give us the highest accuracy overall.

For this model specifically, we decided to implement our own search for the value of K. We used a grid search for the other models, however, the hyperparameter tuning for the KNN model was far too slow--10 parameters would have taken roughly and hour to complete--for us to run each time. Our implementation still produces the best K-valu but is much faster. For the purpose of saving time, we have set the hyperparameter search to false by default. We have run it and included the results in a markdown cell below for convenience and efficiency.

In [ ]:
RUN_KNN_HYPERPARAM_SEARCH = False

In [ ]:
best_k = 0
best_k_accuracy = 0


if RUN_KNN_HYPERPARAM_SEARCH:
    for k in range(1, 10):
        k_folds = 10
        scores = cross_val_score(KNeighborsClassifier(k), train_input_vectors, train_labels, 
                                 cv=k_folds, scoring='f1_weighted')

        average_accuracy = np.mean(scores)

        if average_accuracy > best_k_accuracy:
            best_k_accuracy = average_accuracy
            best_k = k

        print("KNN (k=%d) average accuracy (%d-fold x-val): %f" 
              % (k, k_folds, average_accuracy))


    print("Best k value is %d" % best_k)

```
KNN (k=1) average accuracy (10-fold x-val): 0.912206
KNN (k=2) average accuracy (10-fold x-val): 0.910955
KNN (k=3) average accuracy (10-fold x-val): 0.921259
KNN (k=4) average accuracy (10-fold x-val): 0.932431
KNN (k=5) average accuracy (10-fold x-val): 0.933147
KNN (k=6) average accuracy (10-fold x-val): 0.937488
KNN (k=7) average accuracy (10-fold x-val): 0.929473
KNN (k=8) average accuracy (10-fold x-val): 0.938147
KNN (k=9) average accuracy (10-fold x-val): 0.936519
Best k value is 8
```

<a id='m1_train'></a>

### Training

Now that we have the best K-value, we need to train our model using that ideal parameter. Since we will be using the same code to train and predict for each of the 3 models, we decided to make a function for overall clarity and simplicity.

In [ ]:
def train_and_predict(model, train_input_vectors, train_labels, test_input_vectors):
    model.fit(train_input_vectors, train_labels)
    return model.predict(test_input_vectors)

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 8)
knn_predicted_labels = train_and_predict(knn_model, train_input_vectors, train_labels, test_input_vectors)

### Evaluation

In our evaluation of the models, we checked the accuracy and error rate, the confusion matrix, and the F1 Score using our predicted labels and the test (true) labels. The model evaluation is an important step because it tells us how well our models are performing. We want to ensure that we choose the top performing model out of the 3.

#### Accuracy and Error Rate

Here, we again use a function to print the accuracy and error rates for each of the models to avoid having redundant lines of code in the notebook. 

The classification accuracy is the ratio of the number of correct predictions to the total number of predictions made.

Classification Accuracy = Correct Predictions / Total Predictions

Classification Error = Incorrect Predictions / Total Predictions or 1 - Classification Accuracy

In [ ]:
def print_accuracy_and_error_rate(true_labels, predicted_labels):
    accuracy = accuracy_score(test_labels, predicted_labels)
    print('Accuracy rate: {:0.2f}%'.format(accuracy*100))
    error   = 1 - accuracy
    print('Error rate: {:0.2f}%'.format(error*100))

In [ ]:
print_accuracy_and_error_rate(test_labels, knn_predicted_labels)

We can see that we have a pretty high accuracy rate. This is a decent result, but we want to see if we can do even better with the next model.

#### Confusion Matrix

The confusion matrix measures the performance for the machine learning classification problem. A confusion matrix allows us to calculate our evaluation metrics to determine how well the model performs. Again, we use functions to help cut down on redundant code when plotting each of our confusion matrices.

In [ ]:
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    width, height = cm.shape
    for x in range(width):
        for y in range(height):
            plt.annotate(str(cm[x][y]), xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    

def generate_confusion_matrix(true_labels, predicted_labels):
    all_labels = sorted(set(true_labels).union(set(predicted_labels)))
    cm = confusion_matrix(true_labels, predicted_labels)
    plot_confusion_matrix(cm, all_labels)

In [ ]:
generate_confusion_matrix(test_labels, knn_predicted_labels)

As seen in the category similarity section, business and politics and business and tech are the most similiar categories, so they are naturally the most difficult to tell apart. Here in the confusion matrix, we can see that these categories are the ones causing the most confusion, which makes sense since they are the most similar.

#### F1 Score

The F1 Score allows us to reach a balance between the precision and accuracy of our models. We chose to include the F1 Score along with the accuracy, since it is more useful in the case where there might be an uneven class distribution.

In [ ]:
print('Statistic Report for KNN Classifier')
print(classification_report(test_labels, knn_predicted_labels))

Our precision, recall, and F1 Score results are good as well. They are actually higher than the classification accuracy rating alone, so using the F1 Score was a good choice for us.

## Model #2: Naive Bayes  

### Discussion

A *Naive Bayes Classifier* is a eager learning algorithm which uses probababilities to predict the most likely class for a given example.

To calculate the probability that a document, with features (i.e. TDIDF values) $\{f_1, \ldots , f_n\}$ is in category $c$, we use **Bayes Theorem**. This is given by:
$$ \text{Pr}(c \;|\; \{f_1, \ldots , f_n\}) = \frac{\text{Pr}(\{f_1, \ldots , f_n\} \;|\; c) \ \text{Pr}(c)}{\text{Pr}(\{f_1, \ldots , f_n\})} $$

Since it can be very difficult to calculate $\text{Pr}(\{f_1, \ldots , f_n\} \;|\; c)$, we make an assumption that all the features are **conditionally independent** of each other. This is a rather naive assumption (hence the name Naive Bayes), however it can be rather effective.

This assumption gives us an equation that is much easier to calculate, since now
$$\text{Pr}(\{f_1, \ldots , f_n\} \;|\; c) = \prod_i \text{Pr}(\ f_i \;|\; c)$$

The training stage of the algorithm consists of calculating the components of these equations. When we need to classify an example document, we use its feature values to calculate the probability of the document having each individual category. Then we return the category with the highest probability as our prediction.

There are a number of advantages to using a Naive Bayes classifier, including:
- Good time and memory performance: it's fast and needs little storage
- It can dilute irrelevant features
- It is the optimal solution if the naive assumption is true

However, in some domains, the naive assumption is not true and this can lead to poor performance.

### Hyperparameter Optimization

As we've done before with KNN, we will now use hyperparameter optimization with our Naive Bayes model.

In [ ]:
print('The search range is: ' + str(np.linspace(0.0001,0.0002,11)))
bayes_clf_val = MultinomialNB()
bayes_parameters = {'alpha': (np.linspace(0.0001,0.0002,11))}

In [ ]:
bayes_gs_clf = GridSearchCV(bayes_clf_val, param_grid = bayes_parameters, cv = 10, scoring = 'accuracy')
bayes_gs_clf = bayes_gs_clf.fit(train_input_vectors, train_labels)
NB_parameters = bayes_gs_clf.best_params_
print('The best parameter is: ', NB_parameters['alpha'])

In [ ]:
bayes_gs_clf.grid_scores_

<a id='m2_train'></a>

### Training

Since our best parameter is 0.00011, we will train our model using this value to ensure the best results.

In [ ]:
bayes_model = MultinomialNB(alpha = 0.00011)
bayes_predicted_labels = train_and_predict(bayes_model, train_input_vectors, train_labels, test_input_vectors)

### Evaluation

#### Accuracy and Error Rate

We make use of our function to print the results of the accuracy and error rates for our Naive Bayes model.

In [ ]:
print_accuracy_and_error_rate(test_labels, bayes_predicted_labels)

As we can see here, we have improved our accuracy rate with Naive Bayes compared to KNN. We still have one more model to test, and hopefully we can get even better results.

#### Confusion Matrix

In [ ]:
generate_confusion_matrix(test_labels, bayes_predicted_labels)

Again, we can see that the most similar categories--described in the category similarity section--are the ones causing the most confusion in the matrix.

#### F1 Score

In [ ]:
print('Statistic Report for Naive Bayes Classifier')
print(classification_report(test_labels, bayes_predicted_labels))

The results are even higher with the Naive Bayes model. Our previous model only received a rating of 94%, whereas NB has received a 97% across the board. We will still wait until we evaluate our last model to determine which model is most successful.

## Model #3: Logistic Regression

### Discussion

A *Logistic Regression Classifier* is an eager learning algorithm for predicting a categorical target feature by looking for linear separation between classes. Regular Linear Regression predicts a numeric value, $y$, using a linear combination of input features, $X$. This is given by:

$$ y = b + \sum_{i}^{|X|} w_ix_i $$

where:
- $x_i$ is an input feature, $x_i \in X$ 
- $w_i$ is the weight for that feature
- $b$ is the bias for the function 

Logistic Regression transforms this into a probability using the *Sigmoid Function*, $\phi$:

![Graph of the Sigmoid Function](images/sigmoid.png)

The logistic regression model passes the Linear Regression function into the Sigmoid Function. This generates a result in the range [0, 1]. This corresponds to the probability of the prediction that the document belongs to a certain category, $c$, being true. We choose the most-probable category as our prediction.

$$ prediction(X) = \text{argmax}_c \text{Pr}(X = c) $$

where:
- $x$ is a document
- $C = \{c_i\}$ is the set of possible categories

Logistic Regression can give more informative predictions. As well as predicting a category for a document, the algorithm can also give how confident it is that that prediction is correct. However, it doesn't work very well with non-linearly separable problems, since it relies on a Linear Regression model.

### Hyperparameter Optimization

Based on our previous runs, we've chosen to check 25 parameters from 50 to 75. This range proved to produce the best results when we ran the code prior to this final run. However, we still end up with the same results nonetheless.

In [ ]:
print('The search range is: ' + str(np.linspace(50,75,26)))
log_reg_clf_val = LogisticRegression()
log_reg_parameters = {'C': (np.linspace(50,75,26))}

In [ ]:
log_reg_gs_clf = GridSearchCV(log_reg_clf_val, param_grid = log_reg_parameters, cv = 10, scoring = 'accuracy')
log_reg_gs_clf = log_reg_gs_clf.fit(train_input_vectors, train_labels)
LR_parameters = log_reg_gs_clf.best_params_
print('The best parameter is: ', LR_parameters['C'])

In [ ]:
log_reg_gs_clf.grid_scores_

Since our results didn't vary based on the parameter number, we just decided to use 50 for our training value.

### Training

In [ ]:
log_reg_model = LogisticRegression(C = 50)
log_reg_predicted_labels = train_and_predict(log_reg_model, train_input_vectors, train_labels, test_input_vectors)

### Evaluation

#### Accuracy and Error Rate

In [ ]:
print_accuracy_and_error_rate(test_labels, log_reg_predicted_labels)

Here, we have our highest accuracy rating out of all our models. Based on accuracy alone, logistic regression has proven to be the most successful.

#### Confusion Matrix

In [ ]:
generate_confusion_matrix(test_labels, log_reg_predicted_labels)

Like the two previous models, we can see that business and tech and business and politics are the categories causing the most confusion within the matrix. Again, this is to be expected since they are the most similar categories.

#### F1 Score

In [ ]:
print('Statistic Report for Logistic Regression Classifier')
print(classification_report(test_labels, log_reg_predicted_labels))

Similar to our Naive Bayes model, our logistic regression model has great results for precision, recall, and F1 Score. Like the previous model, it also has a score of 97% across the board; however, logistic regression also received the highest accuracy rating (a 97% as well) out of the all the models, so logistic regression seems like the best choice thus far. We still have a few more steps to verify this before making our final decision.

## Ensembling

Finally, we try creating ensembles from our best models to see if we can push the performance even higher.

An ensemble is a model that is created by combining the output from several different models. For some tasks they can be more effective than a single model on its own, because different models can have different weaknesses, which are counteracted by the other models in the ensemble.

### Voting Ensemble
We have three very effective models, so first we try combining them by creating a voting ensemble. A voting emsemble is a model which makes predictions by running several different models and returning the class that was predicted most often.

In [ ]:
best_classifiers = [
    ('KNN', KNeighborsClassifier(n_neighbors = 8, metric = 'cosine')),
    ('MultinomialNB', MultinomialNB(alpha = 0.0001)),
    ('logistic_regression', LogisticRegression(C = 50)),
]

voting_ensemble = sklearn.ensemble.VotingClassifier(best_classifiers)
voting_ensemble_predicted_labels = train_and_predict(voting_ensemble, train_input_vectors, train_labels, test_input_vectors)
print(classification_report(test_labels, voting_ensemble_predicted_labels))

This voting ensemble is slightly better than naive bayes (0.01 improvement in f1 score). However this difference is probably due to noise, since when we re-run this code the results are sometimes the same as Naive Bayes. For that reason we won't use this ensemble, since it seems to be no better than the base classifier, and it is more slow to run.

### Bagging Ensemble
Next we try a bagging ensemble. A bagging ensemble trains copies of the same model on different random subsets of the train set, then combines them in some way, for example with majority voting. We try it with the Naive Bayes model.

In [ ]:
base_classifier = MultinomialNB(alpha = 0.0001)
bagging_ensemble = sklearn.ensemble.BaggingClassifier(base_classifier)

bagging_ensemble_predicted_labels = train_and_predict(bagging_ensemble, train_input_vectors, train_labels, test_input_vectors)
print(classification_report(test_labels, bagging_ensemble_predicted_labels))

Again this is no more effective than the standard Naive Bayes.

### Bagging Ensemble with Feature Subspacing
A variant of the standard bagging ensemble is bagging with feature subspacing, which trains models on different subsets of input features, rather than different subsets of data.

In [ ]:
base_classifier = MultinomialNB(alpha = 0.0001)
feature_subspace_ensemble = sklearn.ensemble.BaggingClassifier(base_classifier, bootstrap=False, bootstrap_features=True)

feature_subspace_ensemble_predicted_labels = train_and_predict(
    feature_subspace_ensemble, train_input_vectors, train_labels, test_input_vectors)
print(classification_report(test_labels, feature_subspace_ensemble_predicted_labels))

Yet again this is no more effective than the base model.

### Boosting Ensemble
Finally, we try a boosting ensemble. A boosting ensemble trains several models in sequence, where the later models are trained mostly on examples that the earlier models got wrong. By focusing on examples that are usually misclassified, this method aims to counteract a model's weaknesses.

In [ ]:
base_classifier = MultinomialNB(alpha = 0.0001)
boosting_ensemble = sklearn.ensemble.AdaBoostClassifier(base_classifier)
boosting_ensemble_predicted_labels = train_and_predict(
    boosting_ensemble, train_input_vectors, train_labels, test_input_vectors)
print(classification_report(test_labels, boosting_ensemble_predicted_labels))

Finally, this one is also no better.

### Conclusion

None of the ensemble methods meaningfully improved the model score, and all of them are slower to train and run than a single model. For that reason we will not use ensembles for our final model.

<a id='apply_final'></a>

## Applying the Final Model to the Test Set

Retrain our final model on entirety of training set. Include the train split as well as the test split.

In [ ]:
final_model = MultinomialNB(alpha = 0.0001)
# note to Bolu: You can use 'train_and_predict' function here
# just make sure you pass it the correct inputs

Write predicted labels to both a txt file and a csv file.
MM: I image the CSV file is from a dataframe where we read in the test_data and add a column for category using our predictions. The dataframe.to_csv()